# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:

from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
env = UnityEnvironment(file_name='./Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [5]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

KeyboardInterrupt: 

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [5]:
from  collections  import deque
from itertools import count
import time
import torch
from ddpg_agent import Agent
import matplotlib.pyplot as plt
%matplotlib inline

agent = Agent(state_size=state_size, action_size=action_size, random_seed=15)

In [6]:
def ddpg(n_episodes=3000):
    scores_deque = deque(maxlen=100)
    scores_global = []
        
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
        states = env_info.vector_observations                  # get the current state (for each agent)
        scores = np.zeros(num_agents)                          # initialize the score (for each agent)
        agent.reset()
        
        score_average = 0
        timestep = time.time()
        for t in range(3000):
            actions = agent.act(states)
            env_info = env.step(actions)[brain_name]           # send all actions to tne environment
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            agent.step(states, actions, rewards, next_states, dones, t)
            states = next_states                               # roll over states to next time step
            scores += rewards                                  # update the score (for each agent)            
            if np.any(dones):                                  # exit loop if episode finished
                break
        
        score = np.mean(scores)
        scores_deque.append(score)
        score_average = np.mean(scores_deque)
        scores_global.append(score)
        
        print('\rEpisode {}, Average Score: {:.2f}, Max: {:.2f}, Min: {:.2f}, Time: {:.2f}'\
              .format(i_episode, score_average, np.max(scores), np.min(scores), time.time() - timestep), end="\n")        
        
        if i_episode % 10 == 0:
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))   
                       
            
    return scores_global

scores = ddpg()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Episode 1, Average Score: 0.00, Max: 0.00, Min: 0.00, Time: 6.34
Episode 2, Average Score: 0.00, Max: 0.02, Min: 0.00, Time: 9.42
Episode 3, Average Score: 0.05, Max: 0.72, Min: 0.00, Time: 7.60
Episode 4, Average Score: 0.14, Max: 1.31, Min: 0.00, Time: 7.30
Episode 5, Average Score: 0.29, Max: 1.99, Min: 0.21, Time: 7.24
Episode 6, Average Score: 0.37, Max: 1.85, Min: 0.10, Time: 6.61
Episode 7, Average Score: 0.42, Max: 2.09, Min: 0.12, Time: 6.72
Episode 8, Average Score: 0.47, Max: 2.32, Min: 0.23, Time: 6.80
Episode 9, Average Score: 0.51, Max: 2.02, Min: 0.15, Time: 6.90
Episode 10, Average Score: 0.54, Max: 1.51, Min: 0.08, Time: 6.88
Episode 10	Average Score: 0.54
Episode 11, Average Score: 0.54, Max: 1.25, Min: 0.13, Time: 6.82
Episode 12, Average Score: 0.56, Max: 1.45, Min: 0.10, Time: 6.98
Episode 13, Average Score: 0.60, Max: 1.98, Min: 0.13, Time: 7.17
Episode 14, Average Score: 0.62, Max: 1.86, Min: 0.21, Time: 7.34
Episode 15, Average Score: 0.64, Max: 2.21, Min: 0.42,

Episode 117, Average Score: 20.84, Max: 39.44, Min: 33.29, Time: 11.85
Episode 118, Average Score: 21.20, Max: 39.51, Min: 32.10, Time: 11.04
Episode 119, Average Score: 21.55, Max: 39.34, Min: 29.23, Time: 11.09
Episode 120, Average Score: 21.91, Max: 39.65, Min: 32.84, Time: 11.13
Episode 120	Average Score: 21.91
Episode 121, Average Score: 22.25, Max: 39.56, Min: 30.00, Time: 11.22
Episode 122, Average Score: 22.60, Max: 39.46, Min: 32.22, Time: 10.99
Episode 123, Average Score: 22.94, Max: 39.23, Min: 34.86, Time: 11.03
Episode 124, Average Score: 23.28, Max: 38.96, Min: 33.10, Time: 11.17
Episode 125, Average Score: 23.62, Max: 39.60, Min: 34.34, Time: 11.33
Episode 126, Average Score: 23.96, Max: 39.31, Min: 33.65, Time: 10.90
Episode 127, Average Score: 24.30, Max: 39.52, Min: 33.19, Time: 11.13
Episode 128, Average Score: 24.64, Max: 39.60, Min: 29.03, Time: 11.13
Episode 129, Average Score: 24.97, Max: 39.29, Min: 31.40, Time: 11.17
Episode 130, Average Score: 25.32, Max: 39.6

Episode 228, Average Score: 36.87, Max: 39.22, Min: 31.69, Time: 10.52
Episode 229, Average Score: 36.87, Max: 39.64, Min: 35.75, Time: 10.45
Episode 230, Average Score: 36.88, Max: 39.58, Min: 34.15, Time: 10.73
Episode 230	Average Score: 36.88
Episode 231, Average Score: 36.87, Max: 38.42, Min: 34.27, Time: 10.58
Episode 232, Average Score: 36.87, Max: 39.01, Min: 36.23, Time: 10.46
Episode 233, Average Score: 36.89, Max: 39.55, Min: 30.98, Time: 10.67
Episode 234, Average Score: 36.88, Max: 39.58, Min: 33.54, Time: 10.63
Episode 235, Average Score: 36.87, Max: 39.30, Min: 29.23, Time: 10.57
Episode 236, Average Score: 36.86, Max: 39.36, Min: 32.25, Time: 10.72
Episode 237, Average Score: 36.85, Max: 39.41, Min: 30.57, Time: 10.62
Episode 238, Average Score: 36.84, Max: 39.44, Min: 29.37, Time: 10.58
Episode 239, Average Score: 36.83, Max: 39.29, Min: 34.50, Time: 10.64
Episode 240, Average Score: 36.80, Max: 39.25, Min: 27.72, Time: 10.83
Episode 240	Average Score: 36.80
Episode 241

KeyboardInterrupt: 